**Thinkful - 3.2.5 - Drill - Random Forests**

In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
%matplotlib inline

# Replace the path with the correct path for your data.
y2015 = pd.read_csv('data/LoanStats3d.csv',skipinitialspace=True,header=1)

# Note the warning about dtypes.
y2015.head()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


In [78]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
421097
term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
12
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


In [79]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc', 'verification_status_joint'], 1, inplace=True)

# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

In [80]:
pd.get_dummies(y2015).head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,last_credit_pull_d_May-2015,last_credit_pull_d_May-2016,last_credit_pull_d_Nov-2015,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT
0,68009401,72868139.0,16000.0,16000.0,16000.0,14.85,379.39,48000.0,33.18,0.0,...,0,0,0,0,0,0,0,0,1,0
1,68354783,73244544.0,9600.0,9600.0,9600.0,7.49,298.58,60000.0,22.44,0.0,...,0,0,0,0,0,0,0,0,1,0
2,68466916,73356753.0,25000.0,25000.0,25000.0,7.49,777.55,109000.0,26.02,0.0,...,0,0,0,0,0,0,0,0,1,0
3,68466961,73356799.0,28000.0,28000.0,28000.0,6.49,858.05,92000.0,21.60,0.0,...,0,0,0,0,0,0,0,0,1,0
4,68495092,73384866.0,8650.0,8650.0,8650.0,19.89,320.99,55000.0,25.49,0.0,...,0,0,0,0,0,0,0,0,1,0


In [82]:
rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, y, cv=10)

array([ 0.98050391,  0.98007646,  0.98159626,  0.98171499,  0.97594396,
        0.97760627,  0.95039065,  0.98059797,  0.97957584,  0.98045409])

In [87]:
X.drop(['policy_code', 'pymnt_plan_n'], 1, inplace=True)

<bound method BaseEstimator.get_params of SelectKBest(k=10, score_func=<function f_classif at 0x10731ce18>)>


In [101]:
from sklearn.feature_selection import SelectKBest
rfc_kbest = SelectKBest(k=80).fit(X,y)
X_new = rfc_kbest.transform(X)
cross_val_score(rfc, X_new, y, cv=10)

array([ 0.97715561,  0.98002897,  0.98154876,  0.98159626,  0.98026597,
        0.98007599,  0.97998053,  0.98050298,  0.97898212,  0.97912412])

In [102]:
rfc_kbest = SelectKBest(k=20).fit(X,y)
X_new = rfc_kbest.transform(X)
cross_val_score(rfc, X_new, y, cv=10)

array([ 0.96582841,  0.97330864,  0.97639572,  0.98026644,  0.96722869,
        0.97430539,  0.97145503,  0.97800945,  0.97411357,  0.97959911])

In [103]:
rfc_kbest = SelectKBest(k=10).fit(X,y)
X_new = rfc_kbest.transform(X)
cross_val_score(rfc, X_new, y, cv=10)

array([ 0.93324784,  0.92707369,  0.93248795,  0.94478877,  0.90384707,
        0.9382332 ,  0.93673561,  0.94689971,  0.9237419 ,  0.96026695])

In [105]:
rfc_kbest = SelectKBest(k=5).fit(X,y)
X_new = rfc_kbest.transform(X)
cross_val_score(rfc, X_new, y, cv=10)
score = cross_val_score(rfc, X_new, y, cv=10)
print("Weighted Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))

Weighted Accuracy: 0.93 (+/- 0.02)


In [113]:
best = rfc_kbest.get_support(indices=True)
print(best)

[ 16  24 150 165 180]


In [114]:
y2015.columns[best]

IndexError: index 150 is out of bounds for axis 1 with size 102